#### This Sheet contains two models 
#### Frist Model :  Manually selected  features after multiple interation of VIF & P values calculation
#### Second Model: Feature Selected using RFE and then manually tune using VIF & P values calculation

# First Model ( Manual Feature Selection) 

### Import the required libaray for data frame, data visiualization, model selection, data split for training & testing, Linear Regression, stats modeling, calendar

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import calendar
import warnings
warnings.filterwarnings('ignore')

### Load bike rent  csv file data into the bike_df data frame, check the shape, find the numerical data's stats, also check if there is any missing data

In [ ]:
bike_df = pd.read_csv('C:/Asheesh/upgrade/bikeRent/day.csv')
bike_df.head()

In [ ]:
## fine the data frame shape
bike_df.shape

In [ ]:
# find the describption numerical data
bike_df.describe()

In [ ]:
#find the columns data type and if there is any missing value
bike_df.info()

It is good,  there is no missing value from 16 columns. 

### Now find what are the different uniqure values integer columns contains

In [ ]:
# find unique values of weekday column
bike_df['weekday'].unique()

weekday contains unique values 0 to 6.

In [ ]:
# find unique values of workingday column
bike_df['workingday'].unique()

Integer column workingday contains unique values 0 and 1.

In [ ]:
# find unique values of holiday column
bike_df['holiday'].unique()

Integer column holiday contains unique values 0 and 1.

In [ ]:
# find unique values of mnth column
bike_df['mnth'].unique()

Integer column mnth contains unique values 1 and 12.

In [ ]:
# find unique values of season columns
bike_df['season'].unique()

Integer column season contains unique values 1 to 4.

In [ ]:
# find unique values of yr columns
bike_df['yr'].unique()

Integer column yr contains unique values 0 and 1.

In [ ]:
# find unique values of weathersit columns
bike_df['weathersit'].unique()

Integer column weathersit contains unique values 1 to 3.

In [ ]:
bike_df.columns

### Finding the pair plot among the non categorical variables

In [ ]:
sns.pairplot(bike_df[['cnt','temp','atemp','hum','windspeed','casual','registered']])
plt.show()

From the above pair plot we can see there is strong relation between temp & atemp.cloumns
linear relation between cnt and temp, atemp columns
linear relation between cnt and casual, registered column columns

### Heat map amoung the non categorical variables

In [ ]:
sns.heatmap(bike_df[['cnt','temp','atemp','hum','windspeed','casual','registered']].corr(), annot=True)
plt.show()

From the above heatmap  we can see there is strong correlation between cnt, registered and causal columns
Good correlation between cnt, temp and atemp
strong correlation between temp & atemp

### Find out the box plot between cnt and other  categorical variables 

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(2,3,1)
sns.boxplot(x='holiday',y='cnt', data=bike_df)
plt.subplot(2,3,2)
sns.boxplot(x='season',y='cnt', data=bike_df)
plt.subplot(2,3,3)
sns.boxplot(x='mnth',y='cnt', data=bike_df)
plt.subplot(2,3,4)
sns.boxplot(x='weathersit',y='cnt', data=bike_df)
plt.subplot(2,3,5)
sns.boxplot(x='weekday',y='cnt', data=bike_df)
plt.subplot(2,3,6)
sns.boxplot(x='workingday',y='cnt', data=bike_df)

75 percentile for holiday and non-holiday is same. 
For season fall we count goes high.
Form 5th to 10th month, count goes high.
When weather is clear, count goes up and when weather is heavy count goes down. 
Median of the weekday is almost same while 75 percentile go up during the mid of the week. 
Median of working and non-working day is same. 

### Map the categorial variables numerical values with corresponding string values, then create dummy variables

In [ ]:
# bike_df['season'] = bike_df['season'].astype(str)
# bike_df['weathersit'] = bike_df['weathersit'].astype(str)
bike_df['season'] = bike_df['season'].map({1: 'spring', 2:'summer', 3:'fall', 4:'winter'})
bike_df['weathersit'] = bike_df['weathersit'].map({1: 'clear', 2:'mist', 3:'light', 4:'heavy'})
bike_df['mnth'] = bike_df['mnth'].apply(lambda x: calendar.month_abbr[x])
bike_df['weekday'] = bike_df['weekday'].apply(lambda x: calendar.day_abbr[x])

In [ ]:
# create dummy varialbes for categorical variables and drop their first columns as using n-1 columns we can achive the same things.
season = pd.get_dummies(bike_df['season'],dtype='int', drop_first=True)
weathersit = pd.get_dummies(bike_df['weathersit'],dtype='int', drop_first=True)
month = pd.get_dummies(bike_df['mnth'],dtype='int',drop_first=True)
weekday = pd.get_dummies(bike_df['weekday'],dtype='int', drop_first=True)

In [ ]:
# drop variables for which we have created the dummy variables
bike_df = bike_df.drop(['season', 'weathersit', 'mnth', 'weekday'], axis=1)
# drop the registered , casual columns as these are directly giving the cnt. 
# dropping workingday as workingday column contains redundant information as the same info we can achive using holiday and workday columns
# dropping instant & dteday as instant & dteday are numerical values but their mean, median, mode and other stats does not help 
bike_df = bike_df.drop(['registered', 'casual','workingday', 'instant', 'dteday'], axis=1)

In [ ]:
# concatnating dummies to bike_df for our modeling
bike_df=pd.concat([bike_df,season,weathersit,month,weekday], axis=1)

In [ ]:
# checking the all variables including dummies
bike_df.head()

### Spliting the data, standardize the numerical data, then train the model using OLS

In this approach, first I am dropping variable one by one after checking the VIF and p-values. 
Doing iteration in following way 
High VIF & High p value
High VIF & Low p value
Low VIF & High p value

In [ ]:
# split the data set
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
print(df_train.shape)
print(df_test.shape)

In [ ]:
# standardize the numerical data using min max scaler
num_vars=['windspeed', 'cnt','temp', 'atemp', 'hum']
scalar= MinMaxScaler()
df_train[num_vars]=scalar.fit_transform(df_train[num_vars])
df_train[num_vars].head()

In [ ]:
# Train model using OLS
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

R square is 0.85 which is good, but there is variable which have high pvalue too(not very good case)

In [ ]:
# test the current model using the split test data
df_test[num_vars]= scalar.transform(df_test[num_vars])
df_test.head()
y_test=df_test['cnt']
X_test=df_test.drop('cnt',axis=1)
X_test_sm= sm.add_constant(X_test)
y_test_pred= lr_model.predict(X_test_sm)
r2_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
#Check the variance inflation factor for current columns 
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

temp , atemp, hum, spring, winter, summer have high VIF(more than 5), highest p-value among these,  atemp have. So we will drop atemp  

In [ ]:
# drop atemp variable and build model again
bike_df = bike_df.drop([ 'atemp'], axis=1)
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

After removing the atemp, R square is 85. temp, hum, spring, winter, windspeed and summer have high VIF(more than 5), highest VIF temp have So I will drop temp

In [ ]:
bike_df = bike_df.drop([ 'temp'], axis=1)
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

After removing the temp, R square is 82.  hum, spring, winter have high VIF(more than 5), highest VIF hum have So I will drop hum

In [ ]:
bike_df = bike_df.drop([ 'hum'], axis=1)
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

In [ ]:
bike_df = bike_df.drop([ 'winter'], axis=1)
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

In [ ]:
bike_df = bike_df.drop(['spring'], axis=1)
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

After removing the hum, spring, winter, R square is 81.  now VIF of windspeed is more than 05 high VIF. So I will drop windspeed

In [ ]:
bike_df = bike_df.drop([ 'windspeed'], axis=1)
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

After removing high VIF, VIFs are in control. There are some varaibles which have high p-values. So need to remove those varaibles too. Removing the days (other than Mon & Fri days) as these days have high p values 

In [ ]:
bike_df = bike_df.drop([ 'Tue','Wed', 'Thu', 'Sat', 'Sun'], axis=1)
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

Now removing the nov month as it have high p-value

In [ ]:
bike_df = bike_df.drop([ 'Nov'], axis=1)
df_train, df_test = train_test_split(bike_df, test_size=0.3, random_state=100)
y_train=df_train.pop('cnt')
X_train=df_train
X_train_sm= sm.add_constant(X_train)
lr = sm.OLS(y_train,X_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

#### After multiple iteration of remvoing variable and VIF, Now current model's R square is 0.791 and all the variable have very low p value and VIF doing the residual analysis

In [ ]:
# draw the displot of the Residual  
y_pred=lr_model.predict(X_train_sm)
res=y_train-y_pred
sns.displot(res)

Residual is creating a normalized distribution plot

In [ ]:
y_test=df_test['cnt']
X_test=df_test.drop('cnt',axis=1)
X_test_sm= sm.add_constant(X_test)
y_test_pred= lr_model.predict(X_test_sm)
r2_score(y_true=y_test, y_pred=y_test_pred)

#### For the this mdoe on the test data set, the R square is 0.76 which looks pretty fine.

# Second Model( Feature Selection using RFE)

### Now creating the model using the automated feature selection RFE

In this approach, first I am suing the RFE for selection feature and then doing some manual tunining after checking VIF and p values.

In [ ]:
# leading the data in bike_df_ref data frame 
bike_df_ref = pd.read_csv('C:/Asheesh/upgrade/bikeRent/day.csv')
bike_df_ref.head()

In [ ]:
# Mapping category variable then creating the dummy variable, 
# dropping 'season', 'weathersit', 'mnth', 'weekday'
# dropping 'registered', 'casual','workingday', 'instant', 'dteday'
# concatnating dummy varibles to the bike_df_ref datafrom 
bike_df_ref['season'] = bike_df_ref['season'].map({1: 'spring', 2:'summer', 3:'fall', 4:'winter'})
bike_df_ref['weathersit'] = bike_df_ref['weathersit'].map({1: 'clear', 2:'mist', 3:'light', 4:'heavy'})
bike_df_ref['mnth'] = bike_df_ref['mnth'].apply(lambda x: calendar.month_abbr[x])
bike_df_ref['weekday'] = bike_df_ref['weekday'].apply(lambda x: calendar.day_abbr[x])
season = pd.get_dummies(bike_df_ref['season'],dtype='int', drop_first=True)
weathersit = pd.get_dummies(bike_df_ref['weathersit'],dtype='int', drop_first=True)
month = pd.get_dummies(bike_df_ref['mnth'],dtype='int',drop_first=True)
weekday = pd.get_dummies(bike_df_ref['weekday'],dtype='int', drop_first=True)
bike_df_ref = bike_df_ref.drop(['season', 'weathersit', 'mnth', 'weekday'], axis=1)
bike_df_ref = bike_df_ref.drop(['registered', 'casual','workingday', 'instant', 'dteday'], axis=1)
bike_df_ref=pd.concat([bike_df_ref,season,weathersit,month,weekday], axis=1)

In [ ]:
# spliting the the data frame for training and testing, and then standardizing the data. 
df_train, df_test = train_test_split(bike_df_ref, test_size=0.3, random_state=100)
num_vars=['windspeed', 'cnt','temp', 'atemp', 'hum']
scalar= MinMaxScaler()
df_train[num_vars]=scalar.fit_transform(df_train[num_vars])

In [ ]:
# selecting the features using FFE
y_train=df_train.pop('cnt')
X_train=df_train
lm= LinearRegression()
ref= RFE(lm, step=25)
ref.fit(X_train, y_train)

In [ ]:
# Listing the columns and their ranking on the bases of RFE
list(zip(X_train.columns, ref.support_, ref.ranking_))

In [ ]:
col= X_train.columns[ref.support_]
col

In [ ]:
X_train.columns[~ref.support_]

In [ ]:
# Train the model using given reference variables
X_train_ref=df_train[col]
X_train_ref= sm.add_constant(X_train_ref)
lr = sm.OLS(y_train,X_train_ref)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
#find the VIF
vif=pd.DataFrame()
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

In [ ]:
# Test the current model 
df_test[num_vars]= scalar.transform(df_test[num_vars])
df_test.head()
y_test=df_test['cnt']
X_test=df_test[col]
X_test_sm= sm.add_constant(X_test)
y_test_pred= lr_model.predict(X_test_sm)
r2_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
# Drop temp as it has high VIF
col=col.drop('temp')
X_train_ref=df_train[col]
X_train_ref= sm.add_constant(X_train_ref)
lr = sm.OLS(y_train,X_train_ref)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
# Check again VIF of the remaining columns
vif=pd.DataFrame()
X_train =X_train[col]
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

In [ ]:
# Drop hum column and train model again
col=col.drop('hum')
X_train_ref=df_train[col]
X_train_ref= sm.add_constant(X_train_ref)
lr = sm.OLS(y_train,X_train_ref)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
# Chedk VIF
vif=pd.DataFrame()
X_train =X_train[col]
vif['Features']=X_train.columns
vif['VIF']= [variance_inflation_factor(X_train.values,i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

#### Model have 0.83 R square which is good.  P value of the columns are zero or very less. This is my final model.

In [ ]:
# Resudial analysis
y_test=df_test['cnt']
X_test=df_test[col]
X_test_sm= sm.add_constant(X_test)
y_test_pred= lr_model.predict(X_test_sm)
r2_score(y_true=y_test, y_pred=y_test_pred)

#### For this Model, Test data R square is 0.80 which is good. Residual analysis of this

In [ ]:
# draw the displot of the Residual
y_pred=lr_model.predict(X_train_ref)
res=y_train-y_pred
sns.displot(res)
plt.show()

#### Residual is creating a normalized distribution plot